# Vietnamese ASR API — Client Examples

Use this notebook to call your Vietnamese ASR FastAPI server (local or via ngrok).


In [ ]:
# -*- coding: utf-8 -*-
import requests, json, base64, os
from pathlib import Path

# ===========================
# CONFIGURATION
# ===========================
# Update with your public URL from the Colab deployment (or localhost)
API_URL = "http://localhost:8000"  # e.g. 'https://xxxx.ngrok-free.app'

# Optional API key
REQUIRE_API_KEY = os.environ.get('REQUIRE_API_KEY','false').lower()=='true'
API_KEY = os.environ.get('API_KEY', '')
HEADERS = ({'Authorization': f'Bearer {API_KEY}'} if REQUIRE_API_KEY and API_KEY else {})

# Default decoding options
DEFAULT_REPO_ID = 'hynt/sherpa-onnx-zipformer-vi-int8-2025-10-16'
DEFAULT_DECODING_METHOD = 'modified_beam_search'
DEFAULT_NUM_ACTIVE_PATHS = 15

print('='*60)
print('ASR Client configured')
print('API_URL:', API_URL)
print('='*60)


## 1) Health & Ready
Checks the service status.


In [ ]:
print('Health check:')
try:
    r = requests.get(f'{API_URL}/healthz', timeout=10)
    print('GET /healthz ->', r.status_code, r.json())
except Exception as e:
    print('Error:', e)

print('Ready check:')
try:
    r = requests.get(f'{API_URL}/readyz', timeout=20)
    print('GET /readyz  ->', r.status_code, r.json())
except Exception as e:
    print('Error:', e)


## 2) Transcribe (Upload File)
Sends a local file via multipart/form-data to `/v1/transcribe`.


In [ ]:
def transcribe_file(path, decoding_method=DEFAULT_DECODING_METHOD, num_active_paths=DEFAULT_NUM_ACTIVE_PATHS, repo_id=DEFAULT_REPO_ID):
    params = {
        'decoding_method': decoding_method,
        'num_active_paths': num_active_paths,
        'repo_id': repo_id,
    }
    with open(path, 'rb') as f:
        files = {'file': (Path(path).name, f, 'audio/wav')}
        r = requests.post(f'{API_URL}/v1/transcribe', params=params, files=files, headers=HEADERS, timeout=120)
    print('Status:', r.status_code)
    try:
        print(json.dumps(r.json(), ensure_ascii=False, indent=2))
    except Exception:
        print(r.text)
    return r

# Example
# transcribe_file('test_wavs/vietnamese/0.wav')


## 3) Transcribe (From URL)
Downloads audio from a URL and sends JSON with `audio_url`.


In [ ]:
def transcribe_url(audio_url, decoding_method=DEFAULT_DECODING_METHOD, num_active_paths=DEFAULT_NUM_ACTIVE_PATHS, repo_id=DEFAULT_REPO_ID):
    payload = {
        'audio_url': audio_url,
        'decoding_method': decoding_method,
        'num_active_paths': num_active_paths,
        'repo_id': repo_id,
    }
    r = requests.post(f'{API_URL}/v1/transcribe', json=payload, headers={'Content-Type': 'application/json', **HEADERS}, timeout=120)
    print('Status:', r.status_code)
    try:
        print(json.dumps(r.json(), ensure_ascii=False, indent=2))
    except Exception:
        print(r.text)
    return r

# Example
# transcribe_url('https://example.com/audio.wav')


## 4) Transcribe (Base64)
Reads a local file, encodes to base64, sends JSON with `audio_base64`.


In [ ]:
def transcribe_base64(path, decoding_method=DEFAULT_DECODING_METHOD, num_active_paths=DEFAULT_NUM_ACTIVE_PATHS, repo_id=DEFAULT_REPO_ID):
    with open(path, 'rb') as f:
        b64 = base64.b64encode(f.read()).decode('utf-8')
    payload = {
        'audio_base64': b64,
        'decoding_method': decoding_method,
        'num_active_paths': num_active_paths,
        'repo_id': repo_id,
    }
    r = requests.post(f'{API_URL}/v1/transcribe', json=payload, headers={'Content-Type': 'application/json', **HEADERS}, timeout=120)
    print('Status:', r.status_code)
    try:
        print(json.dumps(r.json(), ensure_ascii=False, indent=2))
    except Exception:
        print(r.text)
    return r

# Example
# transcribe_base64('test_wavs/vietnamese/1.wav')


## 5) Batch Transcription
Runs multiple files sequentially.


In [ ]:
def batch_transcribe(paths):
    results = []
    for i, p in enumerate(paths, 1):
        print(f'[{i}/{len(paths)}] {Path(p).name}')
        r = transcribe_file(p)
        try:
            results.append(r.json())
        except Exception:
            results.append({'error': r.text})
    return results

# Example
# batch_transcribe(['test_wavs/vietnamese/0.wav', 'test_wavs/vietnamese/1.wav'])


## 6) Upload in Google Colab
Interactively upload a file in Colab and transcribe it.


In [ ]:
def colab_upload_and_transcribe():
    from google.colab import files
    uploaded = files.upload()
    if not uploaded:
        print('No file uploaded')
        return None
    name = list(uploaded.keys())[0]
    tmp_path = f'/content/{name}'
    with open(tmp_path, 'wb') as f:
        f.write(uploaded[name])
    return transcribe_file(tmp_path)

# Example (uncomment in Colab)
# colab_upload_and_transcribe()


## 7) Quick Start
Minimal end-to-end example.


In [ ]:
# Example quick run
# r = transcribe_file('test_wavs/vietnamese/0.wav')
# if r.ok:
#     print('Text:', r.json().get('text'))
